In [ ]:
%matplotlib inline

In [ ]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob
import re

In [ ]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [ ]:
rad_data = \
pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 10000""",conn)
rad_data.head(5)

In [ ]:
rad_data.shape

### Write a function that returns the impression section of a report

#### Hints

* Not every report will have an impression section
* "INTERPRETATION" and "CONCLUSIONS" might be synonyms for "IMPRESSION"

In [ ]:
def get_impression(report):
    pass


#### In how many reports did we find an impression section?

In [ ]:
def count_impression(report):
    pass


In [ ]:
sum([count_impression(report) for report in rad_data["text"]])

### Define Regular expressions for data cleansing

* Write a regular expression to replace dates in the reports with ``[**DATE**]``
* Write a regular expression to replace times in the reports with ``[**TIME**]``
* Write a regular expression to replaces digits with "d", (e.g. "43 cc" would become "dd cm")

#### Hints

* Look at some sample reports to see what dates and times look like in the reports
* What order would you need to apply the regular expressions?

In [ ]:
for i in range(10):
    print(rad_data.iloc[i]['text'])
    print("*"*42,"\n\n")

In [ ]:
date = re.compile(r"""d""")

In [ ]:
print(rad_data.iloc[0]["text"])

In [ ]:
date.findall(rad_data.iloc[0]["text"])

In [ ]:
digits = re.compile(r"""\d""")

In [ ]:
print(digits.sub("d", rad_data.iloc[0]["text"]))

In [ ]:
rad_data["impression"] = \
rad_data.apply(lambda row: digits.sub("d", get_impression(row["text"])).lower(), axis=1)

In [ ]:
rad_data.head()

### How many unique words occur in the corpus?

#### Hint

1. Use TextBlob
1. Put all the reports into a single string

#### I got 8658

In [ ]:
unique_impression_words = None
len(unique_impression_words)

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
STOPWORDS

In [ ]:
my_stop_words = frozenset(["a", "am", "an", "and", "are", "as", "at", "be", "for", "is", "the", "is", "of", "which", ])

## Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame

### Define a vector space for the radiology corpus

#### Hints

1. How would you build a corpus from words only occuring more than N times?

### Create a new column named `"impression no stops"` where [stop words](https://en.wikipedia.org/wiki/Stop_words) have been dropped from the impression

* The gensim package has stop words defined (``from gensim.parsing.preprocessing import STOPWORDS``

#### Hints
1. Do you agree with dropping all the stop words?
1. How could we create a new stopwords frozen set absent the terms we wan't to keep (double negative?)
1. You could use a regular expressions substitution or token the report first and operate on the list of words.

In [ ]:
rad_data["impression no stops"] = \
rad_data.apply(None, axis=1)

In [ ]:
rad_data.iloc[0]["impression"]

In [ ]:
rad_data.iloc[0]["impression no stops"]

In [ ]:
unique_impression_words = set(TextBlob(" ".join(rad_data["impression no stops"])).words)


In [ ]:
word_map = dict(zip(unique_impression_words,range(len(unique_impression_words))))